# Chain Ladder

Chain Ladder, or CL, is one of the most famous claim reserve method. This method is popular because it use no distribution assumption on claim. Assumption for CL method are: 
1. Cumulative claims, C_${i,j}$ of different accident years $i$ are independent.
2. There exist development factors $f_j > 0, j = 0,1,2,\dots I-1$ such that for all $0 \leq i \leq I$ and $1 \leq j \leq I$
\begin{equation}
E\left[C_{i,j} | C_{i,0}, C_{i,1}, \dots ,C_{i,j-1} \right] = E\left[C_{i,j} | C_{i,j-1} \right] = f_{j-1} C_{i,j-1}
\end{equation}
3. There exist a variance parameters $\sigma^2_j > 0, j = 0,1,2,\dots I-1$ such that for all $0 \leq i \leq I$ and $1 \leq j \leq I$
\begin{equation}
\text{Var}\left(C_{i,j}|C_{i,j-1}\right) = \sigma^2_{j-1} C_{i,j}
\end{equation}

We are not going to proof any of the equation here to keep it light. Reader who are curious in reading the proof, can refer to reference section.

# Notation 

1. $i$ : accident year with $i = 0,1,2,\dots, I$
2. $j$ : development year with $j = 0,1,2,\dots, I$
3. $Y_{i,j}$ : incremental claim from accident year $i$ in development year $j$
4. $C_{i,j}$ : cumulative claim from accident year $i$ in development year $j$

## Packages  

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Preparing Data

We are going to use Worker Compensation Data from CAS Website

In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/leonv1602/claim-reserving/main/data/wkcomp_pos.csv')
data.head()

,GRCODE,GRNAME,AccidentYear,DevelopmentYear,DevelopmentLag,IncurLoss_D,CumPaidLoss_D,BulkLoss_D,EarnedPremDIR_D,EarnedPremCeded_D,EarnedPremNet_D,Single,PostedReserve97_D
0,86,Allstate Ins Co Grp,1988,1988,1,367404,70571,127737,400699,5957,394742,0,281872
1,86,Allstate Ins Co Grp,1988,1989,2,362988,155905,60173,400699,5957,394742,0,281872
2,86,Allstate Ins Co Grp,1988,1990,3,347288,220744,27763,400699,5957,394742,0,281872
3,86,Allstate Ins Co Grp,1988,1991,4,330648,251595,15280,400699,5957,394742,0,281872
4,86,Allstate Ins Co Grp,1988,1992,5,354690,274156,27689,400699,5957,394742,0,281872


In [10]:
df = data[data['GRCODE'] == 7080]
df

,GRCODE,GRNAME,AccidentYear,DevelopmentYear,DevelopmentLag,IncurLoss_D,CumPaidLoss_D,BulkLoss_D,EarnedPremDIR_D,EarnedPremCeded_D,EarnedPremNet_D,Single,PostedReserve97_D
2900,7080,New Jersey Manufacturers Grp,1988,1988,1,167087,41821,65633,196319,607,195712,0,1090093
2901,7080,New Jersey Manufacturers Grp,1988,1989,2,166976,76550,42901,196319,607,195712,0,1090093
2902,7080,New Jersey Manufacturers Grp,1988,1990,3,166458,96697,16353,196319,607,195712,0,1090093
2903,7080,New Jersey Manufacturers Grp,1988,1991,4,170327,112662,10768,196319,607,195712,0,1090093
2904,7080,New Jersey Manufacturers Grp,1988,1992,5,178065,123947,14688,196319,607,195712,0,1090093
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,7080,New Jersey Manufacturers Grp,1997,2002,6,190408,134249,36231,262329,1068,261261,0,1090093
2996,7080,New Jersey Manufacturers Grp,1997,2003,7,186160,140671,29511,262329,1068,261261,0,1090093
2997,7080,New Jersey Manufacturers Grp,1997,2004,8,179091,144865,19742,262329,1068,261261,0,1090093
2998,7080,New Jersey Manufacturers Grp,1997,2005,9,183066,148577,21316,262329,1068,261261,0,1090093


In [14]:
df = df[df['DevelopmentYear'] <= 1997]

In [15]:
tr_inc = pd.pivot_table(df, values='IncurLoss_D', index='AccidentYear',
                       columns=['DevelopmentLag'], aggfunc=np.sum)
tr_inc

DevelopmentLag,1,2,3,4,5,6,7,8,9,10
AccidentYear,,,,,,,,,,
1988,167087.0,166976.0,166458.0,170327.0,178065.0,179126.0,175786.0,176194.0,178076.0,178967.0
1989,179470.0,185511.0,190758.0,202576.0,204649.0,201686.0,202141.0,204552.0,204135.0,NaN
1990,198993.0,215685.0,226829.0,228487.0,225776.0,226678.0,228830.0,228252.0,NaN,NaN
1991,237457.0,245174.0,247201.0,247388.0,245651.0,249402.0,251541.0,NaN,NaN,NaN
1992,256936.0,272664.0,272226.0,266570.0,263346.0,263655.0,NaN,NaN,NaN,NaN
1993,273382.0,273792.0,266742.0,259781.0,261032.0,NaN,NaN,NaN,NaN,NaN
1994,311979.0,290504.0,264756.0,263642.0,NaN,NaN,NaN,NaN,NaN,NaN
1995,288726.0,265868.0,255992.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996,258617.0,236631.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


For Chain Ladder Method we need to change the incremental run off triangle to cumulative run-off triangle

In [16]:
tr_cum = tr_inc.cumsum(axis = 1)
tr_cum

DevelopmentLag,1,2,3,4,5,6,7,8,9,10
AccidentYear,,,,,,,,,,
1988,167087.0,334063.0,500521.0,670848.0,848913.0,1028039.0,1203825.0,1380019.0,1558095.0,1737062.0
1989,179470.0,364981.0,555739.0,758315.0,962964.0,1164650.0,1366791.0,1571343.0,1775478.0,NaN
1990,198993.0,414678.0,641507.0,869994.0,1095770.0,1322448.0,1551278.0,1779530.0,NaN,NaN
1991,237457.0,482631.0,729832.0,977220.0,1222871.0,1472273.0,1723814.0,NaN,NaN,NaN
1992,256936.0,529600.0,801826.0,1068396.0,1331742.0,1595397.0,NaN,NaN,NaN,NaN
1993,273382.0,547174.0,813916.0,1073697.0,1334729.0,NaN,NaN,NaN,NaN,NaN
1994,311979.0,602483.0,867239.0,1130881.0,NaN,NaN,NaN,NaN,NaN,NaN
1995,288726.0,554594.0,810586.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996,258617.0,495248.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Chain Ladder Method  


## Development Factor


\begin{equation} \label{dev}
    f_k = \frac{\sum_{j=1}^{I-k} C_{i+1,j}}{\sum_{j=1}^{I-k} C_{ij}}
\end{equation}

In [21]:
def dev_fac(triangle = tr_cum, i = 0, shape = tr_cum.shape[1]):
    return np.sum(tr_cum.iloc[:shape - i - 1, i+1])/np.sum(tr_cum.iloc[:shape - i -1, i])

In [23]:
dev_factor = pd.DataFrame(data = [dev_fac(i = k) for k in range(9)], index = np.arange(0,9), columns = ['fk'])
dev_factor

,fk
0,1.990867
1,1.493697
2,1.333722
3,1.254411
4,1.205143
5,1.172093
6,1.147747
7,1.129503
8,1.114863


In [24]:
ult_factor = dev_factor.sort_index(ascending=False).cumprod()
ult_factor

,fk
8,1.114863
7,1.259241
6,1.445290
5,1.694015
4,2.041531
3,2.560919
2,3.415555
1,5.101806
0,10.157018


In [25]:
print(f'Total Reserve from Chain Ladder Method {np.array(tr_cum.max(axis = 1)[1:].array).dot(ult_factor-1)[0]}')

Total Reserve from Chain Ladder Method 11666807.583008004


# Reference

Wüthrich, M. V., &amp; Merz, M. (2008). Stochastic Claims Reserving Methods in insurance. Chichester, England: John Wiley &amp; Sons. 